In [1]:
print("hello")
allowed_columns = {'Id'}
input_files = ["Users", "Badges", "Posts", "PostLinks", "Comments", "PostHistory", "Votes", "Tags"]

hello


In [2]:
import xml.etree.ElementTree as ET

# parsing process
tree = ET.parse('askubuntu/Tags.xml')

# the xml document
root = tree.getroot()

print("root is here" , root)

for i in range(10):
    print("attr", root[i].attrib)


root is here <Element 'tags' at 0x0000025E95D03060>
attr {'Id': '1', 'TagName': 'battery', 'Count': '1614', 'ExcerptPostId': '56974', 'WikiPostId': '56973'}
attr {'Id': '2', 'TagName': 'notification', 'Count': '935', 'ExcerptPostId': '60037', 'WikiPostId': '60036'}
attr {'Id': '4', 'TagName': 'software-center', 'Count': '2767', 'ExcerptPostId': '23972', 'WikiPostId': '23971'}
attr {'Id': '6', 'TagName': 'upgrade', 'Count': '9575', 'ExcerptPostId': '31700', 'WikiPostId': '31699'}
attr {'Id': '7', 'TagName': 'live-cd', 'Count': '748', 'ExcerptPostId': '33366', 'WikiPostId': '33365'}
attr {'Id': '8', 'TagName': 'server', 'Count': '16370', 'ExcerptPostId': '31533', 'WikiPostId': '31532'}
attr {'Id': '9', 'TagName': 'ssh', 'Count': '6137', 'ExcerptPostId': '31545', 'WikiPostId': '31544'}
attr {'Id': '10', 'TagName': 'vnc', 'Count': '1083', 'ExcerptPostId': '57694', 'WikiPostId': '57693'}
attr {'Id': '11', 'TagName': 'community', 'Count': '168', 'ExcerptPostId': '29705', 'WikiPostId': '29704

### Save Chunked as small data

In [34]:
import xml.etree.ElementTree as ET

def process_xml_in_chunks(input_file, chunk_size=5, output_prefix="chunked_output", max_chunks=1, xml_file=""):
    """
    Process large XML file in chunks of elements and stop after a certain number of chunks.
    
    :param input_file: Path to the large XML file.
    :param chunk_size: Number of elements to process in each chunk.
    :param output_prefix: Prefix for output file names.
    :param max_chunks: Maximum number of chunks to process.
    """
    context = ET.iterparse(input_file, events=("start", "end"))
    # _, root = next(context)  # Get the root element
    
    chunk_count = 0
    elements_in_chunk = 0
    chunk_root = ET.Element(root.tag)  # Create a new root for each chunk

    for event, elem in context:
        if event == "end" and elem.tag != root.tag:
            # Add element to the current chunk
            chunk_root.append(elem)
            print(chunk_root)
            elements_in_chunk += 1

            # If we've reached the chunk size, save the chunk and reset
            if elements_in_chunk >= chunk_size:
                chunk_count += 1
                save_chunk(chunk_root, output_prefix, chunk_count, xml_file)
                elem.clear()
                
                # Clear the chunk and start over
                chunk_root = ET.Element(root.tag)  # Create a new root for the next chunk
                elements_in_chunk = 0

            # Clear the element from memory

            # Stop after processing the max number of chunks
            if chunk_count >= max_chunks:
                print(f"Reached maximum chunk count ({max_chunks}). Terminating.")
                break

    # If there are any remaining elements in the final chunk, save it
    if elements_in_chunk > 0 and chunk_count < max_chunks:
        chunk_count += 1
        save_chunk(chunk_root, output_prefix, chunk_count)

    print(f"Processed {chunk_count} chunks of {chunk_size} elements each.")

def save_chunk(chunk_root, output_prefix, chunk_count, xml_file):
    output_file = f"small_data/{xml_file}_{output_prefix}_{chunk_count}.xml"
    tree = ET.ElementTree(chunk_root)
    print(tree)
    tree.write(output_file, encoding="utf-8", xml_declaration=True)
    print(f"Saved chunk {chunk_count} to {output_file}")

# Usage example
input_files = ["Users", "Badges", "Posts", "PostLinks", "Comments", "PostHistory", "Votes", "Tags"]
for file in input_files:
    input_file = "askubuntu/" + file + ".xml"  # Path to your large XML file
    process_xml_in_chunks(input_file, chunk_size=100, max_chunks=1, xml_file=file)

<Element 'users' at 0x000002889784EB60>
<Element 'users' at 0x000002889784EB60>
<Element 'users' at 0x000002889784EB60>
<Element 'users' at 0x000002889784EB60>
<Element 'users' at 0x000002889784EB60>
<Element 'users' at 0x000002889784EB60>
<Element 'users' at 0x000002889784EB60>
<Element 'users' at 0x000002889784EB60>
<Element 'users' at 0x000002889784EB60>
<Element 'users' at 0x000002889784EB60>
<Element 'users' at 0x000002889784EB60>
<Element 'users' at 0x000002889784EB60>
<Element 'users' at 0x000002889784EB60>
<Element 'users' at 0x000002889784EB60>
<Element 'users' at 0x000002889784EB60>
<Element 'users' at 0x000002889784EB60>
<Element 'users' at 0x000002889784EB60>
<Element 'users' at 0x000002889784EB60>
<Element 'users' at 0x000002889784EB60>
<Element 'users' at 0x000002889784EB60>
<Element 'users' at 0x000002889784EB60>
<Element 'users' at 0x000002889784EB60>
<Element 'users' at 0x000002889784EB60>
<Element 'users' at 0x000002889784EB60>
<Element 'users' at 0x000002889784EB60>


### DB operation code

In [23]:
# !pip install psycopg2==2.9.9
# !pip install pyyaml==6.0.2

# DB_CONFIG = {
#     'dbname': 'your_database_name',
#     'user': 'your_user',
#     'password': 'your_password',
#     'host': 'your_host',
#     'port': 'your_port'
# }

# def connect():
#     return psycopg2.connect(
#         dbname=DB_CONFIG['dbname'],
#         user=DB_CONFIG['user'],
#         password=DB_CONFIG['password'],
#         host=DB_CONFIG['host'],
#         port=DB_CONFIG['port']
#     )


In [3]:
import psycopg2
import psycopg2.extras as extras
import yaml
import os

def connect():
    config = {}
    # yml_path = os.path.join(os.path.dirname(__file__), './db.yml')
    current_directory = os.getcwd()
    yml_path = os.path.join(current_directory, 'db.yml')
    with open(yml_path, 'r') as file:
        config = yaml.load(file, Loader=yaml.FullLoader)
    return psycopg2.connect(dbname=config['database'],
                            user=config['user'],
                            password=config['password'],
                            host=config['host'],
                            port=config['port'])

def exec_sql_file(path):
    # full_path = os.path.join(os.path.dirname(__file__), f'./{path}')
    current_directory = os.getcwd()
    full_path = os.path.join(current_directory, f'./{path}')
    conn = connect()
    cur = conn.cursor()
    with open(full_path, 'r') as file:
        cur.execute(file.read())
    conn.commit()
    conn.close()

def exec_get_one(sql, args={}):
    conn = connect()
    cur = conn.cursor()
    cur.execute(sql, args)
    one = cur.fetchone()
    conn.close()
    return one

def exec_get_all(sql, args={}):
    conn = connect()
    cur = conn.cursor()
    cur.execute(sql, args)
    # https://www.psycopg.org/docs/cursor.html#cursor.fetchall

    list_of_tuples = cur.fetchall()
    conn.close()
    return list_of_tuples

def exec_commit(sql, args={}):
    conn = connect()
    cur = conn.cursor()
    result = cur.execute(sql, args)
    conn.commit()
    conn.close()
    return result

def execute_df_values(sql, tuples):
    conn = connect()
    cur = conn.cursor()
    result = extras.execute_values(cur, sql, tuples)
    conn.commit()
    conn.close()
    return result

In [4]:
exec_sql_file('create_schema.sql')

### Read and Insert data into DB

In [66]:
### ALL INSERT QUERIES

chunk_size = 3000

users_query = """
    INSERT INTO users (Id, AboutMe, AccountId, CreationDate, DisplayName,
    DownVotes, LastAccessDate, Location, Reputation, UpVotes,
    Views, WebsiteUrl) 
    VALUES %s
    """

badges_query = """
    INSERT INTO badges (Id, Class, Date, Name, TagBased, UserId) 
    VALUES %s
    """

posts_query = """
    INSERT INTO posts (Id, AcceptedAnswerId,AnswerCount,Body,
        ClosedDate,CommentCount,CommunityOwnedDate,ContentLicense,
        CreationDate,FavoriteCount,LastActivityDate,LastEditDate,
        LastEditorDisplayName,LastEditorUserId,OwnerDisplayName,
        OwnerUserId, ParentId,PostTypeId, Score,
        Tags, Title, ViewCount) 
    VALUES %s
    """

post_links_query = """
    INSERT INTO postlinks (Id, CreationDate, LinkTypeId, PostId, RelatedPostId)
    VALUES %s
    """

comments_query = """
    INSERT INTO comments (Id, ContentLicense, CreationDate, PostId, Score,
        Text, UserDisplayName, UserId)
    VALUES %s
    """

post_history_query = """
    INSERT INTO posthistory (Id, Comment, ContentLicense, CreationDate, PostHistoryTypeId,
        PostId, RevisionGUID, Text, UserDisplayName, UserId)
    VALUES %s
    """

votes_query = """
    INSERT INTO votes (Id, BountyAmount, CreationDate, PostId, VoteTypeId)
    VALUES %s
    """

tags_query = """
    INSERT INTO tags (Id, TagName, Count, ExcerptPostId, WikiPostId)
    VALUES %s
    """

In [36]:
def check_foreign_key_exists(table, att_key, att_val):
    # SQL query to check if the userid exists in the users table
    if table.lower() not in {t.lower() for t in input_files} and att_key not in {c.lower() for c in allowed_columns}:
        raise ValueError("Invalid table name")
    sql = "SELECT COUNT(Id) FROM " + table + " WHERE " + att_key + " = %s"
    result = exec_get_one(sql, (att_val,))
    return result[0] > 0

def insert_chunk_into_db(query, chunk_data):
    execute_df_values(query, chunk_data)
    print(f"Inserted {len(chunk_data)} records into the database.")


### INSERTING USERS

In [37]:
import xml.etree.ElementTree as ET
exec_sql_file('create_schema.sql')
#### INSERT INTO USERS

def users_db_chunks(input_file, query, max_chunks=1):
    """
    Process large XML file in chunks of elements and stop after a certain number of chunks.
    
    :param input_file: Path to the large XML file.
    :param chunk_size: Number of elements to process in each chunk.
    :param max_chunks: Maximum number of chunks to process.
    """
    
    context = ET.iterparse(input_file, events=("start", "end"))
    # Skip the root element but start processing children directly
    # _, root = next(context)  # This gets the root but doesn't necessarily process it
    
    chunk_count = 0
    elements_in_chunk = 0
    chunk_data = []  # List to hold data to be inserted into the database

    for event, elem in context:
        if elem.tag == 'row':
            # Collect data from element attributes, handle missing attributes
            if elem.get('Id') is not None:
                element_data = (
                    elem.get('Id'),
                    elem.get('AboutMe'),
                    elem.get('AccountId'),
                    elem.get('CreationDate'),
                    elem.get('DisplayName'),
                    elem.get('DownVotes'),
                    elem.get('LastAccessDate'),
                    elem.get('Location'),
                    elem.get('Reputation'),
                    elem.get('UpVotes'),
                    elem.get('Views'),
                    elem.get('WebsiteUrl')
                )
                chunk_data.append(element_data)
                elements_in_chunk += 1

            # If we've reached the chunk size, save the chunk and reset
            if elements_in_chunk >= chunk_size:
                chunk_count += 1
                insert_chunk_into_db(users_query, chunk_data)
                chunk_data = []  # Reset chunk data
                elements_in_chunk = 0

            elem.clear()

            # Stop after processing the max number of chunks # Remove this code
            # if max_chunks > 0 and chunk_count >= max_chunks:
            #     print(f"Reached maximum chunk count ({max_chunks}). Terminating.")
            #     break

    # If there are any remaining elements in the final chunk, save it
    if elements_in_chunk > 0:
        chunk_count += 1
        insert_chunk_into_db(query, chunk_data)

    print(f"Processed {chunk_count} chunks of {chunk_size} elements each.")

for file in ["Users"]:
    input_file = "askubuntu/" + file + ".xml"  # Path to your large XML file
    users_db_chunks(input_file, users_query, max_chunks=1)


### ISERTING BADGES

In [69]:
def check_foreign_keys_in_chunk(table, ids):
    # Create a query to check the existence of multiple IDs
    formatted_ids = ','.join(map(str, ids))
    sql = f"SELECT COUNT(*) FROM {table} WHERE Id IN ({formatted_ids})"
    result = exec_get_one(sql)
    return result[0]

def check_foreign_key_exists(table, att_key, att_val):
    # SQL query to check if the userid exists in the users table
    if table.lower() not in {t.lower() for t in input_files} and att_key not in {c.lower() for c in allowed_columns}:
        raise ValueError("Invalid table name")
    sql = "SELECT COUNT(Id) FROM " + table + " WHERE " + att_key + " = %s"
    result = exec_get_one(sql, (att_val,))
    return result[0] > 0

def insert_chunk_into_db(query, chunk_data):
    execute_df_values(query, chunk_data)
    print(f"Inserted {len(chunk_data)} records into the database.")



def remove_invalid_entries(chunk_data, valid_user_ids):
    """
    Remove entries with invalid UserIds from chunk_data.
    """
    return [data for data in chunk_data if data[-1] in valid_user_ids]

def extract_ids_from_chunk(chunk_data, loc):
    """
    Extract Ids from chunk_data.
    Make sure to have correct location for the Id in chunk_data.
    """
    return [data[loc] for data in chunk_data]

def check_valid_fk_ids(table, ids):
    """
    Check which ids are valid by querying the database.
    Returns a set of valid ids that are PK in referenced table.
    """
    valid_found_ids = set()
    if ids:
        if table.lower() not in {t.lower() for t in input_files}:
            raise ValueError("Invalid table name")
        sql = "SELECT Id FROM " + table + " WHERE Id IN %s"
        result = exec_get_all(sql, (tuple(ids),))
        valid_found_ids = [row[0] for row in result]
    return valid_found_ids

In [70]:
#### INSERT INTO BADGES
exec_sql_file('create_schema.sql')

def badges_db_chunks(input_file, query, max_chunks=1):
    """
    Process large XML file in chunks of elements and stop after a certain number of chunks.
    
    :param input_file: Path to the large XML file.
    :param max_chunks: Maximum number of chunks to process. MUST REMOVE
    """
    
    context = ET.iterparse(input_file, events=("start", "end"))
    
    # Skip the root element but start processing children directly
    # _, root = next(context)  # This gets the root but doesn't necessarily process it
    
    chunk_count = 0
    elements_in_chunk = 0
    chunk_data = []  # List to hold data to be inserted into the database
    for event, elem in context:
        if elem.tag == 'row':
            # Collect data from element attributes, handle missing attributes
            if elem.get('UserId') is not None:
                element_data = (
                        elem.get('Id'),
                        elem.get('Class'),
                        elem.get('Date'),
                        elem.get('Name'),
                        elem.get('TagBased'),
                        elem.get('UserId')
                )
                chunk_data.append(element_data)
                elements_in_chunk += 1

            # If we've reached the chunk size, save the chunk and reset
            if elements_in_chunk >= chunk_size:
                chunk_count += 1
                user_ids = extract_ids_from_chunk(chunk_data, -1)
                valid_user_ids = check_valid_fk_ids('users', user_ids)
                if len(valid_user_ids) == len(set(user_ids)):
                    # All UserIds are valid, proceed with bulk insertion
                    insert_chunk_into_db(query, chunk_data)
                    chunk_data = []  # Reset chunk data
                    elements_in_chunk = 0
                else:
                    # Some UserIds are invalid, filter out invalid entries
                    valid_chunk_data = remove_invalid_entries(chunk_data, valid_user_ids)
                    insert_chunk_into_db(query, valid_chunk_data)
                    chunk_data = []  # Reset chunk data
                    elements_in_chunk = 0
                        
            elem.clear()

            # Stop after processing the max number of chunks
            # if chunk_count >= max_chunks:
            #     print(f"Reached maximum chunk count ({max_chunks}). Terminating.")
            #     break

    # If there are any remaining elements in the final chunk, save it
    if elements_in_chunk > 0: # and chunk_count < max_chunks:
        chunk_count += 1
        user_ids = extract_ids_from_chunk(chunk_data, -1)
        valid_user_ids = check_valid_fk_ids('users', user_ids)
        if len(valid_user_ids) == len(set(user_ids)):
            # All UserIds are valid, proceed with bulk insertion
            insert_chunk_into_db(query, chunk_data)
        else:
            # Some UserIds are invalid, filter out invalid entries
            valid_chunk_data = remove_invalid_entries(chunk_data, valid_user_ids)
            insert_chunk_into_db(query, valid_chunk_data)

    print(f"Processed {chunk_count} chunks of {chunk_size} elements each.")

for file in ["Badges"]:
    input_file = "askubuntu/" + file + ".xml"  # Path to your large XML file
    badges_db_chunks(input_file, badges_query, max_chunks=5)

Inserted 3000 records into the database.
Inserted 3000 records into the database.
Inserted 3000 records into the database.
Inserted 3000 records into the database.
Inserted 3000 records into the database.
Inserted 3000 records into the database.
Inserted 3000 records into the database.
Inserted 3000 records into the database.
Inserted 3000 records into the database.
Inserted 3000 records into the database.
Inserted 3000 records into the database.
Inserted 3000 records into the database.
Inserted 3000 records into the database.
Inserted 3000 records into the database.
Inserted 3000 records into the database.
Inserted 3000 records into the database.
Inserted 3000 records into the database.
Inserted 3000 records into the database.
Inserted 3000 records into the database.
Inserted 3000 records into the database.
Inserted 3000 records into the database.
Inserted 3000 records into the database.
Inserted 3000 records into the database.
Inserted 3000 records into the database.
Inserted 3000 re

### INSERTING POSTS

In [74]:
#### INSERT INTO USERS
exec_sql_file('create_schema.sql')

def posts_db_chunks(input_file, query, max_chunks=1):
    """
    Process large XML file in chunks of elements and stop after a certain number of chunks.
    
    :param input_file: Path to the large XML file.
    :param chunk_size: Number of elements to process in each chunk.
    :param max_chunks: Maximum number of chunks to process. MUST REMOVE
    """
    
    context = ET.iterparse(input_file, events=("start", "end"))
    
    # Skip the root element but start processing children directly
    # _, root = next(context)  # This gets the root but doesn't necessarily process it
    
    chunk_count = 0
    elements_in_chunk = 0
    chunk_data = []  # List to hold data to be inserted into the database

    for event, elem in context:
        if elem.tag == 'row':
            # Collect data from element attributes, handle missing attributes
            if elem.get('Id') is not None and elem.get('OwnerUserId') is not None:
                element_data = (
                    elem.get('Id'),elem.get('AcceptedAnswerId'),elem.get('AnswerCount'),
                    elem.get('Body'),elem.get('ClosedDate'),elem.get('CommentCount'),
                    elem.get('CommunityOwnedDate'),elem.get('ContentLicense'),
                    elem.get('CreationDate'),elem.get('FavoriteCount'),
                    elem.get('LastActivityDate'),elem.get('LastEditDate'),
                    elem.get('LastEditorDisplayName'),
                    elem.get('LastEditorUserId'),
                    elem.get('OwnerDisplayName'),
                    elem.get('OwnerUserId'),elem.get('ParentId'),
                    elem.get('PostTypeId'),elem.get('Score'),
                    elem.get('Score'),elem.get('Title'),
                    elem.get('ViewCount')
                )
                chunk_data.append(element_data)
                elements_in_chunk += 1

            # If we've reached the chunk size, save the chunk and reset
            if elements_in_chunk >= chunk_size:
                chunk_count += 1
                print(chunk_data)
                user_ids = extract_ids_from_chunk(chunk_data, -7)
                valid_user_ids = check_valid_fk_ids('users', user_ids)
                if len(valid_user_ids) == len(set(user_ids)):
                    # All UserIds are valid, proceed with bulk insertion
                    insert_chunk_into_db(query, chunk_data)
                    chunk_data = []  # Reset chunk data
                    elements_in_chunk = 0
                else:
                    # Some UserIds are invalid, filter out invalid entries
                    valid_chunk_data = remove_invalid_entries(chunk_data, valid_user_ids)
                    insert_chunk_into_db(query, valid_chunk_data)
                    chunk_data = []  # Reset chunk data
                    elements_in_chunk = 0

            elem.clear()

            # Stop after processing the max number of chunks
            if chunk_count >= max_chunks:
                print(f"Reached maximum chunk count ({max_chunks}). Terminating.")
                break

    # If there are any remaining elements in the final chunk, save it
    if elements_in_chunk > 0: # and chunk_count < max_chunks:
        chunk_count += 1
        user_ids = extract_ids_from_chunk(chunk_data, -7)
        valid_user_ids = check_valid_fk_ids('users', user_ids)
        if len(valid_user_ids) == len(set(user_ids)):
            # All UserIds are valid, proceed with bulk insertion
            insert_chunk_into_db(query, chunk_data)
        else:
            # Some UserIds are invalid, filter out invalid entries
            valid_chunk_data = remove_invalid_entries(chunk_data, valid_user_ids)
            insert_chunk_into_db(query, valid_chunk_data)

    print(f"Processed {chunk_count} chunks of {chunk_size} elements each.")

for file in ["Posts"]:
    input_file = "askubuntu/" + file + ".xml"  # Path to your large XML file
    posts_db_chunks(input_file, posts_query, max_chunks=1)


[('56973', None, None, '<p>Maybe <a href="http://linux.aldeby.org/get-rid-of-your-battery-may-be-broken-notification.html">these</a> instructions will help you to get rid of that message.</p>\n\n<p>Added instructions from the link, <kbd>Alt</kbd>+<kbd>F2</kbd>, then type in <code>gconf-editor</code>. </p>\n\n<p>Navigate to <code>/apps/gnome-power-manager/notify/low_capacity</code> and untick the value.</p>\n\n<p>Or a single command:</p>\n\n<pre><code>gconftool --set /apps/gnome-power-manager/notify/low_capacity --type boolean false\n</code></pre>\n', None, '0', None, 'CC BY-SA 3.0', '2010-07-28T19:15:30.357', None, '2012-07-30T13:51:42.147', '2012-07-30T13:51:42.147', None, '2', None, '4', '1', '2', '41', '41', None, None)]
[('56973', None, None, '<p>Maybe <a href="http://linux.aldeby.org/get-rid-of-your-battery-may-be-broken-notification.html">these</a> instructions will help you to get rid of that message.</p>\n\n<p>Added instructions from the link, <kbd>Alt</kbd>+<kbd>F2</kbd>, then

### INSERTING POST LINKS

In [45]:
#### INSERT INTO USERS
exec_sql_file('create_schema.sql')

def post_links_db_chunks(input_file, query, max_chunks=1):
    """
    Process large XML file in chunks of elements and stop after a certain number of chunks.
    
    :param input_file: Path to the large XML file.
    :param chunk_size: Number of elements to process in each chunk.
    :param max_chunks: Maximum number of chunks to process. MUST REMOVE
    """
    
    context = ET.iterparse(input_file, events=("start", "end"))
    
    # Skip the root element but start processing children directly
    # _, root = next(context)  # This gets the root but doesn't necessarily process it
    
    chunk_count = 0
    elements_in_chunk = 0
    chunk_data = []  # List to hold data to be inserted into the database

    for event, elem in context:
        if elem.tag == 'row':
            # Collect data from element attributes, handle missing attributes
            if elem.get('PostId') is not None and elem.get('RelatedPostId') is not None:
                element_data = (
                    elem.get('Id'),elem.get('CreationDatePostId'),
                    elem.get('LinkTypeId'),elem.get('CommentCount'),
                    elem.get('RelatedPostId')
                )
                chunk_data.append(element_data)
                elements_in_chunk += 1

            # If we've reached the chunk size, save the chunk and reset
            if elements_in_chunk >= chunk_size:
                chunk_count += 1
                insert_chunk_into_db(query, chunk_data)
                chunk_data = []  # Reset chunk data
                elements_in_chunk = 0

            elem.clear()

            # Stop after processing the max number of chunks
            # if chunk_count >= max_chunks:
            #     print(f"Reached maximum chunk count ({max_chunks}). Terminating.")
            #     break

    # If there are any remaining elements in the final chunk, save it
    if elements_in_chunk > 0: # and chunk_count < max_chunks:
        chunk_count += 1
        insert_chunk_into_db(query, chunk_data)

    print(f"Processed {chunk_count} chunks of {chunk_size} elements each.")

for file in ["PostLinks"]:
    input_file = "small_data/" + file + ".xml"  # Path to your large XML file
    post_links_db_chunks(input_file, post_links_query, max_chunks=1)

Inserted 4 records into the database.
Processed 1 chunks of 100 elements each.


### INSERTING COMMENTS

In [47]:
#### INSERT INTO USERS
exec_sql_file('create_schema.sql')

def comments_db_chunks(input_file, query, max_chunks=1):
    """
    Process large XML file in chunks of elements and stop after a certain number of chunks.
    
    :param input_file: Path to the large XML file.
    :param chunk_size: Number of elements to process in each chunk.
    :param max_chunks: Maximum number of chunks to process. MUST REMOVE
    """
    
    context = ET.iterparse(input_file, events=("start", "end"))
    
    # Skip the root element but start processing children directly
    # _, root = next(context)  # This gets the root but doesn't necessarily process it
    
    chunk_count = 0
    elements_in_chunk = 0
    chunk_data = []  # List to hold data to be inserted into the database

    for event, elem in context:
        if elem.tag == 'row':
            # Collect data from element attributes, handle missing attributes
            if elem.get('PostId') is not None and elem.get('UserId') is not None:
                element_data = (
                    elem.get('Id'),elem.get('ContentLicense'),
                    elem.get('CreationDate'),elem.get('PostId'),
                    elem.get('Score'),elem.get('Text'),
                    elem.get('UserDisplayName'),elem.get('UserId')
                )
                chunk_data.append(element_data)
                elements_in_chunk += 1

            # If we've reached the chunk size, save the chunk and reset
            if elements_in_chunk >= chunk_size:
                chunk_count += 1
                insert_chunk_into_db(query, chunk_data)
                chunk_data = []  # Reset chunk data
                elements_in_chunk = 0

            elem.clear()

            # Stop after processing the max number of chunks
            # if chunk_count >= max_chunks:
            #     print(f"Reached maximum chunk count ({max_chunks}). Terminating.")
            #     break

    # If there are any remaining elements in the final chunk, save it
    if elements_in_chunk > 0: # and chunk_count < max_chunks:
        chunk_count += 1
        insert_chunk_into_db(query, chunk_data)

    print(f"Processed {chunk_count} chunks of {chunk_size} elements each.")


for file in ["Comments"]:
    input_file = "small_data/relations_enabeld/" + file + ".xml"  # Path to your large XML file
    comments_db_chunks(input_file, comments_query, max_chunks=1)

Inserted 6 records into the database.
Processed 1 chunks of 100 elements each.


### INSERTING POST HISTORY

In [50]:
#### INSERT INTO USERS
exec_sql_file('create_schema.sql')

def post_hist_db_chunks(input_file, query, max_chunks=1):
    """
    Process large XML file in chunks of elements and stop after a certain number of chunks.
    
    :param input_file: Path to the large XML file.
    :param chunk_size: Number of elements to process in each chunk.
    :param max_chunks: Maximum number of chunks to process. MUST REMOVE
    """
    
    context = ET.iterparse(input_file, events=("start", "end"))
    
    # Skip the root element but start processing children directly
    # _, root = next(context)  # This gets the root but doesn't necessarily process it
    
    chunk_count = 0
    elements_in_chunk = 0
    chunk_data = []  # List to hold data to be inserted into the database

    for event, elem in context:
        if elem.tag == 'row':
            # Collect data from element attributes, handle missing attributes
            if elem.get('PostId') is not None and elem.get('UserId') is not None:
                element_data = (
                    elem.get('Id'),elem.get('Comment'),
                    elem.get('ContentLicense'),elem.get('CreationDate'),
                    elem.get('PostHistoryTypeId'),
                    elem.get('PostId'),elem.get('RevisionGUID'),
                    elem.get('Text'),elem.get('UserDisplayName'),
                    elem.get('UserId')
                )
                chunk_data.append(element_data)
                elements_in_chunk += 1

            # If we've reached the chunk size, save the chunk and reset
            if elements_in_chunk >= chunk_size:
                chunk_count += 1
                insert_chunk_into_db(query, chunk_data)
                chunk_data = []  # Reset chunk data
                elements_in_chunk = 0

            elem.clear()

            # Stop after processing the max number of chunks
            # if chunk_count >= max_chunks:
            #     print(f"Reached maximum chunk count ({max_chunks}). Terminating.")
            #     break

    # If there are any remaining elements in the final chunk, save it
    if elements_in_chunk > 0: # and chunk_count < max_chunks:
        chunk_count += 1
        insert_chunk_into_db(query, chunk_data)

    print(f"Processed {chunk_count} chunks of {chunk_size} elements each.")

for file in ["PostHistory"]:
    input_file = "small_data/" + file + ".xml"  # Path to your large XML file
    post_hist_db_chunks(input_file, post_history_query, max_chunks=1)

Inserted 8 records into the database.
Processed 1 chunks of 100 elements each.


### INSERTING VOTES

In [52]:
#### INSERT INTO USERS
exec_sql_file('create_schema.sql')

def votes_db_chunks(input_file, query, max_chunks=1):
    """
    Process large XML file in chunks of elements and stop after a certain number of chunks.
    
    :param input_file: Path to the large XML file.
    :param chunk_size: Number of elements to process in each chunk.
    :param max_chunks: Maximum number of chunks to process. MUST REMOVE
    """
    
    context = ET.iterparse(input_file, events=("start", "end"))
    
    # Skip the root element but start processing children directly
    # _, root = next(context)  # This gets the root but doesn't necessarily process it
    
    chunk_count = 0
    elements_in_chunk = 0
    chunk_data = []  # List to hold data to be inserted into the database

    for event, elem in context:
        if elem.tag == 'row':
            # Collect data from element attributes, handle missing attributes
            if elem.get('PostId') is not None:
                element_data = (
                    elem.get('Id'),elem.get('BountyAmount'),
                    elem.get('CreationDate'),elem.get('PostId'),
                    elem.get('VoteTypeId')
                )
                chunk_data.append(element_data)
                elements_in_chunk += 1

            # If we've reached the chunk size, save the chunk and reset
            if elements_in_chunk >= chunk_size:
                chunk_count += 1
                insert_chunk_into_db(query, chunk_data)
                chunk_data = []  # Reset chunk data
                elements_in_chunk = 0

            elem.clear()

            # Stop after processing the max number of chunks
            # if chunk_count >= max_chunks:
            #     print(f"Reached maximum chunk count ({max_chunks}). Terminating.")
            #     break

    # If there are any remaining elements in the final chunk, save it
    if elements_in_chunk > 0: # and chunk_count < max_chunks:
        chunk_count += 1
        insert_chunk_into_db(query, chunk_data)

    print(f"Processed {chunk_count} chunks of {chunk_size} elements each.")

for file in ["Votes"]:
    input_file = "small_data/" + file + ".xml"  # Path to your large XML file
    votes_db_chunks(input_file, votes_query, max_chunks=1)

Inserted 4 records into the database.
Processed 1 chunks of 100 elements each.


### INSERTING TAGS

In [53]:
#### INSERT INTO USERS
exec_sql_file('create_schema.sql')

def tags_db_chunks(input_file, query, max_chunks=1):
    """
    Process large XML file in chunks of elements and stop after a certain number of chunks.
    
    :param input_file: Path to the large XML file.
    :param chunk_size: Number of elements to process in each chunk.
    :param max_chunks: Maximum number of chunks to process. MUST REMOVE
    """
    
    context = ET.iterparse(input_file, events=("start", "end"))
    
    # Skip the root element but start processing children directly
    # _, root = next(context)  # This gets the root but doesn't necessarily process it
    
    chunk_count = 0
    elements_in_chunk = 0
    chunk_data = []  # List to hold data to be inserted into the database

    for event, elem in context:
        if elem.tag == 'row':
            # Collect data from element attributes, handle missing attributes
            if elem.get('ExcerptPostId') is not None and elem.get('WikiPostId'):
                element_data = (
                    elem.get('Id'),elem.get('TagName'),
                    elem.get('Count'),elem.get('ExcerptPostId'),
                    elem.get('WikiPostId')
                )
                chunk_data.append(element_data)
                elements_in_chunk += 1

            # If we've reached the chunk size, save the chunk and reset
            if elements_in_chunk >= chunk_size:
                chunk_count += 1
                insert_chunk_into_db(query, chunk_data)
                chunk_data = []  # Reset chunk data
                elements_in_chunk = 0

            elem.clear()

            # Stop after processing the max number of chunks
            # if chunk_count >= max_chunks:
            #     print(f"Reached maximum chunk count ({max_chunks}). Terminating.")
            #     break

    # If there are any remaining elements in the final chunk, save it
    if elements_in_chunk > 0: # and chunk_count < max_chunks:
        chunk_count += 1
        insert_chunk_into_db(query, chunk_data)

    print(f"Processed {chunk_count} chunks of {chunk_size} elements each.")

for file in ["Tags"]:
    input_file = "small_data/" + file + ".xml"  # Path to your large XML file
    tags_db_chunks(input_file, tags_query, max_chunks=1)

Inserted 2 records into the database.
Processed 1 chunks of 100 elements each.


### GENERAL CODE TO INSERT DATA IN CHUNCKS

In [8]:
def process_xml_in_chunks(file, input_file, chunk_size=5, max_chunks=1):
    """
    Process large XML file in chunks of elements and stop after a certain number of chunks.
    
    :param input_file: Path to the large XML file.
    :param chunk_size: Number of elements to process in each chunk.
    :param max_chunks: Maximum number of chunks to process.
    """
    
    context = ET.iterparse(input_file, events=("end",)) # need to include start, end
    _, root = next(context)  # Get the root element, need to skip this
    
    chunk_count = 0
    elements_in_chunk = 0
    chunk_data = []  # List to hold data to be inserted into the database

    for event, elem in context:
        if elem.tag == 'row':
            # Collect data from element attributes, handle missing attributes
            element_data = (
                safe_int(elem.get('Id')),
                elem.get('TagName'),
                safe_int(elem.get('Count')),
                safe_int(elem.get('ExcerptPostId')),
                safe_int(elem.get('WikiPostId'))
            )
            chunk_data.append(element_data)
            elements_in_chunk += 1

            # If we've reached the chunk size, save the chunk and reset
            if elements_in_chunk >= chunk_size:
                chunk_count += 1
                insert_chunk_into_db(chunk_data)
                chunk_data = []  # Reset chunk data
                elements_in_chunk = 0

            elem.clear()

            # Stop after processing the max number of chunks
            # if chunk_count >= max_chunks:
            #     print(f"Reached maximum chunk count ({max_chunks}). Terminating.")
            #     break

    # If there are any remaining elements in the final chunk, save it
    if elements_in_chunk > 0: # and chunk_count < max_chunks:
        chunk_count += 1
        insert_chunk_into_db(chunk_data)

    print(f"Processed {chunk_count} chunks of {chunk_size} elements each.")

def safe_int(value):
    """Convert value to integer or return None if the value is None or cannot be converted."""
    try:
        return int(value) if value is not None else None
    except (ValueError, TypeError):
        return None

def insert_chunk_into_db(chunk_data):
    sql_insert = """
    INSERT INTO tags (id, TagName, Count, ExcerptPostId, WikiPostId) 
    VALUES %s
    """
    execute_df_values(sql_insert, chunk_data)
    print(f"Inserted {len(chunk_data)} records into the database.")
for file in input_files:
    input_file = "small_data/" + file + ".xml"  # Path to your large XML file
    process_xml_in_chunks(file, input_file, chunk_size=100, max_chunks=1)

ForeignKeyViolation: insert or update on table "tags" violates foreign key constraint "tags_excerptpostid_fkey"
DETAIL:  Key (excerptpostid)=(60037) is not present in table "posts".


### Get all keys as output by reading over the file

In [3]:
import xml.etree.ElementTree as ET
import os

def extract_keys_from_xml(file_path):
    keys = set()
    try:
        context = ET.iterparse(file_path, events=("end",))
        # _, root = next(context)  # Get the root element
        for event, elem in context:
            if elem.tag == 'row':
                # Update the set with attribute names
                keys.update(elem.attrib.keys())
                elem.clear()
    except Exception as e:
        print(f"Error processing file {file_path}: {e}")
    return keys

for file in input_files:
    file_path = "askubuntu/" + file + ".xml"
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print(file_path)
    unique_keys = extract_keys_from_xml(file_path)
    print("\nUnique attribute keys found:")
    for key in sorted(unique_keys):
        print(key)
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    


### Max length and keys

In [10]:
import xml.etree.ElementTree as ET

def extract_keys_and_max_lengths(file_path):
    # Dictionary to store each key with its max length, longest value, and count of nulls
    key_info = {}

    try:
        context = ET.iterparse(file_path, events=("end",))
        # _, root = next(context)  # Get the root element
        total_records = 0
        for event, elem in context:
            if elem.tag == 'row':
                total_records = total_records + 1
                # For each attribute in the current element
                for key, value in elem.attrib.items():
                    value_length = len(value)
                    
                    # If the key is not already in the dictionary, initialize it
                    if key not in key_info:
                        key_info[key] = {
                            "max_length": value_length,
                            "longest_value": value,
                            "null_count": 0  # Initialize null count
                        }
                    else:
                        # Update max length and longest value if current value is longer
                        if value_length > key_info[key]["max_length"]:
                            key_info[key]["max_length"] = value_length
                            key_info[key]["longest_value"] = value
                
                # Check for missing/empty values (nulls)
                for key in key_info:
                    if key not in elem.attrib or elem.attrib[key] == "":
                        key_info[key]["null_count"] += 1

                elem.clear()  # Clear the element from memory

    except Exception as e:
        print(f"Error processing file {file_path}: {e}")
    print("Total Records: ", total_records)
    return key_info


for file in input_files:
    # File path to your large XML file
    file_path = "askubuntu/" + file + ".xml"
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print(file_path)

    # Extract the keys and their max lengths, longest values, and null counts
    key_info = extract_keys_and_max_lengths(file_path)

    # Print the results
    print("\nKey information with max value lengths, longest values, and null counts:")
    for key, info in sorted(key_info.items()):
        print(f"Key: {key}")
        print(f"  Max length: {info['max_length']}")
        print(f"  Longest value: {info['longest_value']}")
        print(f"  Null count: {info['null_count']}\n")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++")


++++++++++++++++++++++++++++++++++++++++++++++++++++++++
askubuntu/Users.xml
Total Records:  1450497

Key information with max value lengths, longest values, and null counts:
Key: AboutMe
  Max length: 7126
  Longest value: <p>I am developing with</p>
<ul>
<li><a href="/questions/tagged/python" class="post-tag" title="show questions tagged &#39;python&#39;" aria-label="show questions tagged &#39;python&#39;" rel="tag" aria-labelledby="tag-python-tooltip-container">python</a> to automate stuff and make work easier</li>
<li><a href="/questions/tagged/django" class="post-tag" title="show questions tagged &#39;django&#39;" aria-label="show questions tagged &#39;django&#39;" rel="tag" aria-labelledby="tag-django-tooltip-container">django</a> because it brings a lot of things together
<ul>
<li>Also <a href="/questions/tagged/flask" class="post-tag" title="show questions tagged &#39;flask&#39;" aria-label="show questions tagged &#39;flask&#39;" rel="tag" aria-labelledby="tag-flask-tooltip-con

In [6]:
# Tesiting one file

import xml.etree.ElementTree as ET

def extract_keys_and_max_lengths(file_path):
    try:
        context = ET.iterparse(file_path, events=("end",))
        # _, root = next(context)  # Get the root element
        total_records = 0
        for event, elem in context:
            if elem.tag == 'row':
                total_records = total_records + 1
                elem.clear()  # Clear the element from memory

    except Exception as e:
        print(f"Error processing file {file_path}: {e}")
    print("Total Records: ", total_records)
    return key_info


for file in ["Tags"]:
    # File path to your large XML file
    file_path = "askubuntu/" + file + ".xml"
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print(file_path)

    # Extract the keys and their max lengths, longest values, and null counts
    extract_keys_and_max_lengths(file_path)


++++++++++++++++++++++++++++++++++++++++++++++++++++++++
askubuntu/Tags.xml
Total Records:  3155


### Q5, Fail with Transaction

In [24]:
def get_comments():
    return exec_get_all("SELECT * FROM comments")

def test_transaction():
    try:
        conn = connect()
        cur = conn.cursor()
        
        # Insert first row
        print("Insert first row")
        cur.execute("INSERT INTO comments (Id, ContentLicense, CreationDate, PostId, Score, Text, UserDisplayName, UserId) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)",
                    (10000001, 'CC BY-SA 2.5', '2010-07-28T19:36:59.773', 56973, 5, 'Great post!', 'user1', 5))
        
        print("Insert second row")
        # Insert second row with error (non-existing UserId)
        cur.execute("INSERT INTO comments (Id, ContentLicense, CreationDate, PostId, Score, Text, UserDisplayName, UserId) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)",
                    (10000002, 'CC BY-SA 2.5', '2010-07-28T19:36:59.773', 56973, 5, 'Another post!', 'user2', 50))
        
        print("Insert third row")
        # Insert third row
        cur.execute("INSERT INTO comments (Id, ContentLicense, CreationDate, PostId, Score, Text, UserDisplayName, UserId) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)",
                    (10000003, 'CC BY-SA 2.5', '2010-07-30T19:36:59.773', 56973, 5, 'Nice post!', 'user1', 5))
                    
        
        conn.commit()  # Commit transaction
    except Exception as e:
        conn.rollback()  # Rollback on error
        print(f"Transaction failed: {e}")
    finally:
        conn.close()

initial_state = get_comments()
print("Comments Before Transaction: ", len(initial_state))
insert_with_error()
final_state = get_comments()
print("Comments After Transaction: ", len(final_state))

if initial_state == final_state:
    print("Transaction rolled back successfully. Database state is unchanged.")
else:
    print("Database state has changed, transaction did not roll back properly.")


Comments Before Transaction:  6
Insert first row
Insert second row
Transaction failed: insert or update on table "comments" violates foreign key constraint "comments_userid_fkey"
DETAIL:  Key (userid)=(50) is not present in table "users".

Comments After Transaction:  6
Transaction rolled back successfully. Database state is unchanged.
